<a href="https://colab.research.google.com/github/uditbhaskar21/PySpark/blob/main/GrandSlamWinner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install pyspark


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 2.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.1-py2.py3-none-any.whl size=311285387 sha256=b1fd5e0fd3db6859e1255864e8a19165cb6d13760b2cccfb94f796c06795a964
  Stored in directory: /root/.cache/pip/wheels/0d/77/a3/ff2f74cc9ab41f8f594dabf0579c2a7c6de920d584206e0834
Successfully built pyspark


In [4]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("example").getOrCreate()
print(spark.version)

3.4.1


In [5]:
from pyspark.sql.functions import *

In [7]:
players_data = [
    (1, 'Nadal'),
    (2, 'Federer'),
    (3, 'Novak')
]
player_col = ['id','name']
player_df = spark.createDataFrame(players_data,player_col)

In [8]:
player_df.show()

+---+-------+
| id|   name|
+---+-------+
|  1|  Nadal|
|  2|Federer|
|  3|  Novak|
+---+-------+



In [9]:
c_data = [
    (2017,2,1,1,2),
    (2018,3,1,3,2),
    (2019,3,1,1,3)
]

c_col = ['year','Wimbeldon','French','US','Aus']
c_df = spark.createDataFrame(c_data,c_col)
c_df.show()

+----+---------+------+---+---+
|year|Wimbeldon|French| US|Aus|
+----+---------+------+---+---+
|2017|        2|     1|  1|  2|
|2018|        3|     1|  3|  2|
|2019|        3|     1|  1|  3|
+----+---------+------+---+---+



# PySpark Approach

In [12]:
df1 = (c_df.select("year",col("Wimbeldon").alias("player")).unionAll(c_df.select("year",col("French").alias("player"))).unionAll(c_df.select("year",col("US").alias("player")))
.unionAll(c_df.select("year",col("Aus").alias("player"))))

df1.show()


+----+------+
|year|player|
+----+------+
|2017|     2|
|2018|     3|
|2019|     3|
|2017|     1|
|2018|     1|
|2019|     1|
|2017|     1|
|2018|     3|
|2019|     1|
|2017|     2|
|2018|     2|
|2019|     3|
+----+------+



In [13]:
df2 = df1.groupBy("player").agg(count("*").alias("grand_slams_count"))
df2.show()

+------+-----------------+
|player|grand_slams_count|
+------+-----------------+
|     2|                3|
|     3|                4|
|     1|                5|
+------+-----------------+



In [15]:
result_df = df2.join(player_df,df2.player == player_df.id, "inner").select("id","name","grand_slams_count").orderBy("id")
result_df.show()

+---+-------+-----------------+
| id|   name|grand_slams_count|
+---+-------+-----------------+
|  1|  Nadal|                5|
|  2|Federer|                3|
|  3|  Novak|                4|
+---+-------+-----------------+



# **SQL**

In [11]:
player_df.createOrReplaceGlobalTempView("Players")
c_df.createGlobalTempView("Champ")

Split rows

In [ ]:
select year,wimbeldon as  player from cham
union all
select year,french as  player from cham
union all
select year,US as  player from cham
union all
select year,aus as  player from cham


Aggregation

In [ ]:
with cte as(
    select year,wimbeldon as  player from cham
union all
select year,french as  player from cham
union all
select year,US as  player from cham
union all
select year,aus as  player from cham

)

select player, count(1) as grand_slam_count from cte
group by player

Getting player name

In [ ]:
with cte as(
    select year,wimbeldon as  player from cham
union all
select year,french as  player from cham
union all
select year,US as  player from cham
union all
select year,aus as  player from cham

),
with cte2 as(
select player, count(1) as grand_slam_count from cte
group by player)

select player_id,player_name, grand_slam_count from cte2 join players on cte2.player = players.player_id

# **Aproach2**